# XML



Тип данных <b>```xml```</b>.


In [1]:
%%time 
declare @xml xml
set @xml = '<root/>'
select @xml
go

set nocount on

declare @xml xml
set @xml = '<root> </root>'
select @xml
go

declare @xml xml
set @xml = 'Text'
select @xml
go

declare @xml xml
set @xml = '<root><element Attr="Text" /></root>'
select @xml
go

declare @xml xml
set @xml = '<root><element Attr=''Text'' /></root>'
select @xml
go

declare @xml xml
set @xml = '
<root>
	<element Attr=''Text1'' />
	<element Attr=''Text2''>
		<subelement>
			Text3
		</subelement>
	</element>
</root>
'
select @xml
go

(1 row affected)

Total execution time: 00:00:00.0062984

Commands completed successfully.

Total execution time: 00:00:00.0037364

Commands completed successfully.

...

(No column name)
<root />


(No column name)
Text


(No column name)
"<root><element Attr=""Text"" /></root>"


(No column name)
"<root><element Attr=""Text"" /></root>"


(No column name)
"<root><element Attr=""Text1"" /><element Attr=""Text2""><subelement> Text3 </subelement></element></root>"


Метод query() (XPath как подмножество XQuery):


In [0]:

declare @xml xml
set @xml = '
<root>
	<element Attr=''Text1'' />
	<element Attr=''Text2''>
		<subelement>
			Text3
		</subelement>
	</element>
</root>
'
select @xml.query('//element/subelement')
go

Метод exist():

In [0]:

declare @xml xml
set @xml = '
<root>
	<element Attr=''Text1'' />
	<element Attr=''Text2''>
		<subelement>
			Text3
		</subelement>
	</element>
</root>
'
select @xml, @xml.exist('//element/subelement')
go


Загрузка данных (требуются права на выполнение openrowset):

In [2]:
use tempdb
go

create table #swimming_results (
	id          int     not null identity, 
	xml_value   xml 
)

declare @xml    xml

set @xml = (
    select *    
    from openrowset (bulk 'https://github.com/timoti1/T-SQL/blob/master/SQL/samples/SwimmingCompetition.xml', single_blob)  q
)

select @xml.query('/competitions/row/tournament')

--select @xml.value('(/row/result)[1]/athlete_name','varchar(100)') 
go

Commands completed successfully.

Total execution time: 00:00:00.0031117

Commands completed successfully.

Total execution time: 00:00:00.4281143

Использование FLOWR для трансформации XML документа (еще одно подмножество языка XQuery):

In [0]:
declare @xml xml
set @xml = (select * from openrowset (bulk 'samples/SwimmingCompetition.xml', single_blob)  q)

select @xml.query(
'
<doc>
{
	for $Person in /root/Person
	where $Person/@City = "Seattle"
	return
	<Person FirstName="{$Person/Name/FirstName}">
	<City>{ $Person/@City }</City>
	<Address>{ $Person/@Address }</Address>
	</Person>
}
</doc>
'
)
go

Тот же код, но используя внешний параметр/переменную: